# Using Google Geocoding API(webservice) from Oracle

### To call any webservice, need to make http call which in oracle is done by utl_http.request package.

#### Below are the steps to access Google Geocoding webservices from Oracle.



###### 1. We've to create an ACL (Access Control List) in oracle first, below is the code for the same.


In [ ]:

begin

-- Create the ACL first 

dbms_network_acl_admin.create_acl (
    acl         => 'utl_http_map.xml', -- name of acl could be anything
    description => 'HTTP Access',      -- Description
    principal   => 'VIBHU',            -- principal will be the schema name you want to call the webservice from
    is_grant    => TRUE,
    privilege   => 'connect'           -- connect & resource privileges can be provided to the ACL
  );
  
 -- assign the host which you want to access throught the ACL
 
  dbms_network_acl_admin.assign_acl (
    acl        => 'utl_http_map.xml',          -- name of the acl will be the same as the above one
    host       => '*maps.googleapis.com*',     -- host will be the base URL of the host 
    lower_port => 25,                          -- put the lower_port as 25 if you keep it null it will contact at default 80 
    upper_port => NULL                         -- at this port oracle admin service is already to it may throw an error
  );
  
  commit;
end;


##### 2. Have to create oracle wallet to keep the google certificates, for services without certificates you can directly access the URL without using                  wallet.

###### First go the Oracle installed BIN path and check ORAPKI.jar is present,
       G:\app\Orauser\product\12.1.0\dbhome_1\BIN

###### Create a directory e.g. wallet and then execute the below command to make that directory an oracle wallet
       orapki wallet create -wallet G:\app\Orauser\product\12.1.0\dbhome_1\BIN\wallet       
       It will prompt for a passord 2 times this is where you set the wallet password set it and remember it.

In [21]:
%%html
<img src ="images/oracle_wallet_created.PNG",width=500,height=200>

##### Once the wallet is created add the certificates which you can download from [api.google.com](api.google.com) in .p7b format. Once you have            it paste it in the wallet directory. that's not it you have to now add(install) this certificate in oracle wallet as well. use the below command.                 Password will be what you set in just above
     orapki wallet add -wallet G:\app\Orauser\product\12.1.0\dbhome_1\BIN\wallet  -cert G:\app\Orauser\product\12.1.0\dbhome_1\BIN\wallet\google_maps.p7b -trusted_cert -pwd password -jsafe
     
##### Likewise add all the 3 certificates from google mentioned below.

In [32]:
%%html
<img src ="images/google_certs.PNG",width=100,height=10>

##### You can list all the installed certificates by using below command
      orapki wallet display -wallet . -pwd password  -jsafe
##### Refer below screenshot to learn more(masked password for security)
      
Congrates if you don't get any error your oracle wallet has been created successfully with all the certificates installed in it.

In [33]:
%%html
<img src ="images/adding_p7b_cert.PNG",width=100,height=10>

##### Its time to check using the below query whether all these setup is working or not

In [ ]:
select
utl_http.request
      ( 'https://maps.googleapis.com/maps/api/geocode/json?address=' ||
        utl_url.escape('1600 Amphitheatre Parkway, Mountain View, CA&key=<your_api_key_should_be_added_here>')
        ,NULL,'file:G:\app\Orauser\product\12.1.0\dbhome_1\BIN\wallet','urmila24') from dual;

In [35]:
%%html
<img src ="images/query_result.PNG",width=100,height=100>

In [36]:
%%html
<img src ="images/query_result1.PNG",width=100,height=100>

It will be similar if you execute the same URL from your browser

In [38]:
%%html
<img src ="images/browser_result.PNG",width=100,height=100>